In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# Set the URL to be scraped
url = "https://www.flipkart.com/search?q=mens+shirt&sid=clo%2Cash%2Caxc%2Cmmk%2Ckp7&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&as-pos=2&as-type=RECENT&suggestionId=mens+shirt%7CCasual+Shirts&requestId=633f77af-1180-41e4-896b-00db898b4e57&as-backfill=on&sort=popularity"

# Send a GET request to the URL
r = requests.get(url)
print(r)

<Response [200]>


In [3]:
# Create a BeautifulSoup object to parse the HTML content of the response
soup = BeautifulSoup(r.text, "lxml")

In [4]:
# Extract URL of pages 1-10
page1 = url
pages_url = [page1]

for i in range(1, 10):
    new_page = soup.find_all("a", class_ = "ge-49M")[i]['href']
    pages_url.append("https://www.flipkart.com" + new_page)
    
len(pages_url)

10

In [5]:
# Extract URL of pages 11-20
for i in range(0, 10):
    url = "https://www.flipkart.com/search?q=mens+shirt&sid=clo%2Cash%2Caxc%2Cmmk%2Ckp7&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_7_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=mens+shirt%7CCasual+Shirts&requestId=633f77af-1180-41e4-896b-00db898b4e57&sort=popularity&page=15"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    new_page = soup.find_all("a", class_ = "ge-49M")[i]['href']
    pages_url.append("https://www.flipkart.com" + new_page)

len(pages_url)

20

In [21]:
# Create a empty datafram
df = pd.DataFrame()

# Add empty columns to the DataFrame
df["Brand"] = []
df["Product"] = []
df["Price"] = []
df["Rating"] = []
df["Total_ratings"] = []
df["Total_reviews"] = []
df["Fit"] = []
df["Pattern"] = []

In [23]:
# Initialize a variable to keep track of the page count
page_count = 0

# Iterate over the pages_url list
for j in pages_url:
    page_count+=1
    
    # Print the current page being processed
    print("working on page", page_count)
    
    # Set the URL to the current page
    url = j
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    
    # Find all the <a> tags with class "IRpwTa" that contain the titles
    title_code = soup.find_all("a", class_ = "IRpwTa")

    # Iterate over the titile_code list
    for i in title_code:
        # Extract the URL from the <a> tag and construct the complete URL
        new_url = "https://www.flipkart.com" + i['href']
        new_r = requests.get(new_url)
        new_soup = BeautifulSoup(new_r.text, "lxml")
        
        # Try to find the attributed using the specified class, if not found, set it to "NA"
        try:
            brand = new_soup.find_all("span", class_ = "G6XhRU")[0].text
        except:
            brand = "NA"
        
        try:
            product = new_soup.find_all("span", class_ = "B_NuCI")[0].text
        except:
            product = "NA"
        
        try:
            price = new_soup.find_all("div", class_ = "_30jeq3 _16Jk6d")[0].text[1:]
        except:
            price = "NA"
        

        try:
            rating = new_soup.find_all("div", class_ = "_3LWZlK _3uSWvT")[0].text
        except:
            rating = "NA"

        try:
            ratings_reviews = new_soup.find_all("span", class_ = "_2_R_DZ")[0].text.split()
            total_ratings = ratings_reviews[0]
            total_reviews = ratings_reviews[3]
        except:
            total_ratings = "NA"
            total_reviews = "NA"

        try:
            fit = product.split()[1]
        except:
            fit = "NA"
            
        try:
            pattern = product.split()[3]
        except:
            pattern = "NA"

        new_row = {'Brand': brand, 'Product': product, 'Price': price, 'Rating': rating,
                   'Total_ratings': total_ratings, 'Total_reviews': total_reviews,
                   'Fit': fit, 'Pattern': pattern}

        # Create a new DataFrame with the new row
        new_df = pd.DataFrame([new_row])

        # Concatenate the new DataFrame with the original DataFrame
        df = pd.concat([df, new_df], ignore_index=True)

working on page 1
working on page 2
working on page 3
working on page 4
working on page 5
working on page 6
working on page 7
working on page 8
working on page 9
working on page 10
working on page 11
working on page 12
working on page 13
working on page 14
working on page 15
working on page 16
working on page 17
working on page 18
working on page 19
working on page 20


In [26]:
# Check the dataframe
df

,Brand,Product,Price,Rating,Total_ratings,Total_reviews,Fit,Pattern
0,Qugue,Men Regular Fit Printed Cut Away Collar Casual...,299,4.1,"2,225",130,Regular,Printed
1,Try This,Women Slim Fit Graphic Print Mandarin Collar C...,389,3.6,"8,396",289,Slim,Graphic
2,FUBAR,Men Slim Fit Striped Spread Collar Casual Shirt,279,3.8,"1,161",85,Slim,Striped
3,VeBNoR,Men Regular Fit Printed Spread Collar Casual S...,289,4.1,"10,783","1,010",Regular,Printed
4,Voroxy,Men Regular Fit Solid Spread Collar Casual Shirt,309,3.7,"22,948","1,996",Regular,Solid
...,...,...,...,...,...,...,...,...
1471,WROGN,Men Regular Fit Solid Casual Shirt,"1,376",NA,NA,NA,Regular,Solid
1472,PROVOGUE,Men Slim Fit Checkered Casual Shirt,574,4.1,14,1,Slim,Checkered
1473,Oosh,Men Regular Fit Solid Casual Shirt,279,3.8,24,2,Regular,Solid
1474,Allen Solly,Men Slim Fit Checkered Casual Shirt,959,4.4,48,1,Slim,Checkered


In [27]:
# Save the DataFrame as a CSV file
df.to_csv("flipkart_data.csv", index=False)